In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [6]:
if is_jupyter: 
    seed        = 43
    num_patients = 5
    num_providers = 5
    provider_capacity = 1
    noise = 0
    fairness_constraint = -1
    num_trials = 10
    utility_function = "semi_synthetic_comorbidity"
    order="uniform"
    online_arrival = False  
    new_provider = True 
    out_folder = "policy_comparison"
    average_distance = 20.2
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=1)
    parser.add_argument('--noise', help='Noise in theta', type=float, default=0.1)
    parser.add_argument('--average_distance', help='Maximum distance patients are willing to go', type=float, default=20.2)
    parser.add_argument('--fairness_constraint', help='Maximum difference in average utility between groups', type=float, default=-1)
    parser.add_argument('--num_trials', help='Number of trials', type=int, default=100)
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument("--online_arrival",action="store_true",help="Patients arrive one-by-one")
    parser.add_argument("--new_provider",action="store_true",help="Are we simulating a new provider matching")
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    num_trials = args.num_trials
    noise = args.noise
    average_distance = args.average_distance
    fairness_constraint = args.fairness_constraint
    provider_capacity = args.provider_capacity
    utility_function = args.utility_function
    order = args.order
    online_arrival = args.online_arrival
    new_provider = args.new_provider 
    out_folder = args.out_folder
    
assert not(online_arrival and new_provider)
save_name = secrets.token_hex(4)  

In [7]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'utility_function': utility_function, 
        'order': order, 
        'num_trials': num_trials, 
        'noise': noise, 
        'average_distance': average_distance,
        'online_arrival': online_arrival,
        'new_provider': new_provider,
        'fairness_constraint': fairness_constraint} 

## Baselines

In [8]:
seed_list = [seed]
restrict_resources()

In [46]:
policy = one_shot_policy
per_epoch_function = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['random_num_matches'])/num_patients,np.mean(results['random_patient_utilities'])))

random policy
Matches 0.5, Utilities 0.49967781998878197


In [21]:
policy = one_shot_policy
per_epoch_function = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy policy
Matches 1.0, Utilities 0.7696011012838758


In [22]:
policy = one_shot_policy
if fairness_constraint != -1:
    per_epoch_function = get_fair_optimal_policy(fairness_constraint,seed)
else:
    per_epoch_function = optimal_policy
name = "omniscient_optimal"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

omniscient_optimal policy
Matches 1.0, Utilities 0.775


## Optimization-Based

In [23]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

lp policy
Matches 1.0, Utilities 0.775


In [12]:
policy = one_shot_policy
per_epoch_function = gradient_policy
name = "gradient_descent"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

for key in rewards:
    results['{}_{}'.format(name,key)] = rewards[key]
print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

gradient_descent policy
Iter 0: Obj = 0.8668, Worst = 0.4508, AdvSteps = 10
Iter 10: Obj = 0.9366, Worst = 0.5527, AdvSteps = 9
Iter 20: Obj = 0.9819, Worst = 0.6241, AdvSteps = 8
Iter 30: Obj = 1.0025, Worst = 0.6715, AdvSteps = 7
Early stopping at iteration 40 (improvement: 0.0086)
Iter 0: Obj = 1.1248, Worst = 0.6411, AdvSteps = 10
Iter 10: Obj = 1.2263, Worst = 0.7487, AdvSteps = 9
Iter 20: Obj = 1.2892, Worst = 0.8228, AdvSteps = 8
Iter 30: Obj = 1.3171, Worst = 0.8602, AdvSteps = 7
Iter 40: Obj = 1.3288, Worst = 0.8957, AdvSteps = 6
Early stopping at iteration 50 (improvement: 0.0054)
Iter 0: Obj = 1.1237, Worst = 0.6401, AdvSteps = 10
Iter 10: Obj = 1.2254, Worst = 0.7478, AdvSteps = 9
Iter 20: Obj = 1.2887, Worst = 0.8224, AdvSteps = 8
Iter 30: Obj = 1.3169, Worst = 0.8601, AdvSteps = 7
Iter 40: Obj = 1.3287, Worst = 0.8956, AdvSteps = 6
Early stopping at iteration 50 (improvement: 0.0055)
Iter 0: Obj = 1.1920, Worst = 0.7110, AdvSteps = 10
Iter 10: Obj = 1.2904, Worst = 0.8108

KeyboardInterrupt: 

## Save Data

In [40]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)